<a href="https://colab.research.google.com/github/hardeybisey/neural-network/blob/main/traingpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-05-06 18:15:19--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2023-05-06 18:15:20 (30.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



<a href='https://arxiv.org/pdf/1706.03762.pdf'> Attention Paper </a>


In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

In [3]:
text = open('input.txt', 'r', encoding='utf-8').read()
print("length of dataset in characters: ", len(text))
print(text[:200])

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [4]:
chars = sorted(list(set(text)))
# create a mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [5]:
learning_rate = 2e-4
block_layers = 6
n_head = 6
dropout = 0.25
block_size = 256
n_emb = 360
vocab_size = len(chars)
batch_size = 32
eval_iters = 200
eval_interval = 5000
data = torch.tensor(encode(text), dtype=torch.long)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n = int(0.9*len(data))

In [6]:
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    """Helper function to get mini bactches"""
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    """Helper function for calculating loss"""
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [38]:
class Head(nn.Module):
    """ one head of self attention"""
    def __init__(self, head_size, masked=True):
        super().__init__()
        self.key = nn.Linear(n_emb, head_size, bias=False)
        self.query = nn.Linear(n_emb, head_size, bias=False)
        self.value = nn.Linear(n_emb, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size )))
        self.masked = masked
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) # (B,T,C)
        q = self.query(x) # (B,T,C)
        v = self.value(x) # (B,T,C)
        # compute attention scores, mask it and normalize.
        score = q @ k.transpose(-2,-1) * (C**-0.5)# (B,T,C) @ (B,C,T) -> (B,T,T)
        if self.masked:
          score = score.masked_fill(self.tril[:T,:T] == 0, float('-inf')) # (B,T,T)
        score = F.softmax(score, dim = -1)
        score = self.dropout(score)
        # perform the weighted aggregation of the values
        out = score @ v # (B,T,T) @ (B,T,C) -> (B,T,C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self attention """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.projection = nn.Linear(n_emb,n_emb)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
      out = torch.cat([head(x) for head in self.heads], dim=-1)  # (B,T,C)
      out = self.projection(out)
      out = self.dropout(out)
      return out

class FeedForward(nn.Module):
    """ feed forward network """
    def __init__(self, n_emb):
        super().__init__()
        self.network = nn.Sequential(nn.Linear(n_emb, 4 * n_emb), 
                                     nn.ReLU(),
                                     nn.Linear(4 * n_emb, n_emb),
                                     nn.Dropout(dropout))
        
    def forward(self, x):
      return self.network(x)

class Block(nn.Module):
    def __init__(self, n_emb, n_head):
        super().__init__()
        head_size = n_emb // n_head
        self.satt = MultiHeadAttention(n_head, head_size)
        self.ff = FeedForward(n_emb)
        self.ln1 = nn.LayerNorm(n_emb)
        self.ln2 = nn.LayerNorm(n_emb)

    def forward(self, x):
        out = x + self.satt(self.ln1(x))
        out = out + self.ff(self.ln2(out))
        return out

In [39]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        self.lm_head = nn.Linear(n_emb, vocab_size)
        self.blocks = nn.Sequential(*[Block(n_emb, n_head=n_head) for _ in range(block_layers)])
        self.ln = nn.LayerNorm(n_emb)
            
        
    def forward(self, idx, targets=None):
        B , T = idx.shape
        token_emb = self.token_embedding_table(idx) #(B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T,C)
        x = token_emb + pos_emb  #(B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B , T , C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits , loss
      
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:,-block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:,-1,:] # (B, C) focus on only the last time step 
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx,idx_next), dim=1) #append new sampled index to the running context
        return idx

In [40]:
model = BigramLanguageModel()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

9.492545 M parameters


In [47]:
max_iter = 0
for iter in range(max_iter):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb , yb = get_batch('train')
    logit , loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

# saving and loading for inference 

In [ ]:
# torch.save(model.state_dict(), PATH)

# # loading

# model = BigramLanguageModel() #instantiate model arcitecture
# model.load_state_dict(torch.load(PATH))

# saving and loading to continue training

In [ ]:
# # saving
# torch.save({'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict()}, PATH)

# # loading
# checkpoint = torch.load(PATH, map_location=device)
# model = BigramLanguageModel() #instantiate model arcitecture
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3) #instantiate optimizer
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])